# เกม หรือ การค้นหาแบบมีคู่ต่อสู้ (GAMES OR ADVERSARIAL SEARCH)

โน้ตบุ๊คนี้ใช้เป็นเนื้อหาสนับสนุนสำหรับหัวข้อที่ครอบคลุมใน **บทที่ 5 - การค้นหาแบบมีคู่ต่อสู้** ในหนังสือ *Artificial Intelligence: A Modern Approach.* โน้ตบุ๊คนี้ใช้การดำเนินงานจากโมดูล [games.py](https://github.com/aimacode/aima-python/blob/master/games.py) มาเรียกใช้ class, method, ตัวแปรสากล ฯลฯ ที่ต้องการจากโมดูล games

# สารบัญ (CONTENTS)

* การแทนค่าเกม (Game Representation)
* ตัวอย่างเกม (Game Examples)
    * Tic-Tac-Toe
    * เกมรูปที่ 5.2 (Figure 5.2 Game)
* Min-Max
* Alpha-Beta
* ผู้เล่น (Players)
* มาเล่นเกมกัน! (Let's Play Some Games!)

In [1]:
from games import *
from notebook import psource, pseudocode

# การแทนค่าเกม (GAME REPRESENTATION)

ในการแทนค่าเกม เราจะใช้ class `Game` ซึ่งเราสามารถสร้าง subclass และเขียนฟังก์ชันของมันใหม่เพื่อแทนค่าเกมของเราเอง เครื่องมือช่วยคือ namedtuple `GameState` ซึ่งในบางกรณีจะมีประโยชน์มาก โดยเฉพาะเมื่อเกมของเราต้องการให้เราจำกระดาน (เช่น หมากรุก)

## namedtuple `GameState`

`GameState` เป็น [namedtuple](https://docs.python.org/3.5/library/collections.html#collections.namedtuple) ที่แทนค่าสถานะปัจจุบันของเกม มันถูกใช้เพื่อช่วยแทนค่าเกมที่มีสถานะไม่สามารถแทนค่าได้ง่ายๆ แบบปกติ หรือสำหรับเกมที่ต้องการความจำของกระดาน เช่น Tic-Tac-Toe

`Gamestate` ถูกกำหนดดังนี้:

`GameState = namedtuple('GameState', 'to_move, utility, board, moves')`

* `to_move`: แทนค่าคิวของใครที่จะเดินต่อไป

* `utility`: เก็บค่าประโยชน์ของสถานะเกม การเก็บค่าประโยชน์นี้เป็นไอเดียที่ดี เพราะเมื่อคุณทำ Minimax Search หรือ Alphabeta Search คุณจะสร้างการเรียกซ้ำหลายครั้ง ซึ่งจะเดินทางไปจนถึงสถานะสิ้นสุด เมื่อการเรียกซ้ำเหล่านี้กลับขึ้นไปยังผู้เรียกเดิม เราได้คำนวณค่าประโยชน์สำหรับสถานะเกมหลายสถานะแล้ว เราจึงเก็บค่าประโยชน์เหล่านี้ใน `GameState` ที่เกี่ยวข้องเพื่อหลีกเลี่ยงการคำนวณซ้ำทั้งหมดอีกครั้ง

* `board`: dict ที่เก็บกระดานของเกม

* `moves`: เก็บรายการของการเดินที่ถูกกฎหมายที่เป็นไปได้จากตำแหน่งปัจจุบัน

## class `Game`

มาดู class `Game` ในโมดูลของเราเรา เราจะเห็นว่ามันมีฟังก์ชัน ได้แก่ `actions`, `result`, `utility`, `terminal_test`, `to_move` และ `display`

เราจะเห็นว่าฟังก์ชันเหล่านี้ยังไม่ได้ถูกดำเนินการจริง class นี้เป็นเพียง template class เรากำลังจะสร้าง class สำหรับเกมของเรา โดยการสืบทอด class `Game` นี้และการดำเนินการ method ทั้งหมดที่กล่าวถึงใน `Game`

In [2]:
%psource Game

ตอนนี้มาดูรายละเอียดของ method ทั้งหมดใน class `Game` ของเรา คุณต้องดำเนินการ method เหล่านี้เมื่อคุณสร้าง class ใหม่ที่จะแทนค่าเกมของคุณ

* `actions(self, state)`: เมื่อได้รับสถานะเกม method นี้จะสร้างการกระทำที่ถูกกฎหมายทั้งหมดที่เป็นไปได้จากสถานะนี้ เป็น list หรือ generator การส่งคืน generator แทน list มีข้อดีคือประหยัดพื้นที่และคุณยังสามารถดำเนินการกับมันเหมือน list


* `result(self, state, move)`: เมื่อได้รับสถานะเกมและการเดิน method นี้จะส่งคืนสถานะเกมที่คุณได้จากการเดินในสถานะเกมนี้


* `utility(self, state, player)`: เมื่อได้รับสถานะเกมสิ้นสุดและผู้เล่น method นี้จะส่งคืนค่าประโยชน์สำหรับผู้เล่นนั้นในสถานะเกมสิ้นสุดที่กำหนด ขณะที่ดำเนินการ method นี้ ให้สมมติว่าสถานะเกมเป็นสถานะเกมสิ้นสุด ตรรกะในโมดูลนี้เป็นเช่นนั้น method นี้จะถูกเรียกเฉพาะในสถานะเกมสิ้นสุด


* `terminal_test(self, state)`: เมื่อได้รับสถานะเกม method นี้ควรส่งคืน `True` ถ้าสถานะเกมนี้เป็นสถานะสิ้นสุด และ `False` ในกรณีอื่น


* `to_move(self, state)`: เมื่อได้รับสถานะเกม method นี้จะส่งคืนผู้เล่นที่จะเล่นต่อไป ข้อมูลนี้มักจะถูกเก็บไว้ในสถานะเกม ดังนั้น method นี้ทำหน้าที่เพียงแยกข้อมูลนี้และส่งคืน


* `display(self, state)`: method นี้พิมพ์/แสดงสถานะปัจจุบันของเกม

# ตัวอย่างเกม (GAME EXAMPLES)

ด้านล่างเราจะให้ตัวอย่างสำหรับเกมที่คุณสามารถสร้างและทดลองใช้งานได้

## Tic-Tac-Toe

ดู class `TicTacToe` method ทั้งหมดที่กล่าวถึงใน class `Game` ได้ถูกดำเนินการที่นี่

In [3]:
%psource TicTacToe

class `TicTacToe` ได้รับการสืบทอดมาจาก class `Game` ดังที่กล่าวไว้ก่อนหน้า คุณจริงๆ อยากจะทำแบบนี้ การตรวจจับบั๊กและข้อผิดพลาดจะง่ายขึ้นมาก

method เพิ่มเติมใน TicTacToe:

* `__init__(self, h=3, v=3, k=3)`: เมื่อคุณสร้าง class ที่สืบทอดมาจาก class `Game` (class `TicTacToe` ในกรณีของเรา) คุณจะต้องสร้าง object ของ class ที่สืบทอดนี้เพื่อเริ่มต้นเกม การเริ่มต้นนี้อาจต้องการข้อมูลเพิ่มเติมบางอย่างที่จะถูกส่งไปยัง `__init__` เป็นตัวแปร สำหรับกรณีของเกม `TicTacToe` ของเรา ข้อมูลเพิ่มเติมนี้จะเป็นจำนวนแถว `h` จำนวนคอลัมน์ `v` และจำนวน X หรือ O ที่ต่อเนื่องกันในแถว คอลัมน์ หรือแนวทแยงที่ต้องการเพื่อชนะ `k` นอกจากนี้ สถานะเกมเริ่มต้นต้องถูกกำหนดที่นี่ใน `__init__`


* `compute_utility(self, board, move, player)`: method ในการคำนวณค่าประโยชน์ของเกม TicTacToe หาก 'X' ชนะด้วยการเดินนี้ method นี้จะส่งคืน 1; หาก 'O' ชนะจะส่งคืน -1; อื่นๆ ส่งคืน 0


* `k_in_row(self, board, move, player, delta_x_y)`: method นี้จะส่งคืน `True` หากมีการสร้างเส้นบนกระดาน TicTacToe ด้วยการเดินล่าสุด มิฉะนั้น `False`

### TicTacToe GameState

ตอนนี้ ก่อนที่เราจะเริ่มดำเนินการเกม `TicTacToe` เราต้องตัดสินใจว่าเราจะแทนค่าสถานะเกมของเราอย่างไร โดยทั่วไป สถานะเกมจะให้ข้อมูลปัจจุบันทั้งหมดเกี่ยวกับเกมในเวลาใดก็ได้ เมื่อคุณได้รับสถานะเกม คุณควรจะสามารถบอกได้ว่าถึงคิวใครต่อไป เกมจะมีลักษณะอย่างไรบนกระดานจริง (หากมี) ฯลฯ สถานะเกมไม่จำเป็นต้องรวมประวัติของเกม หากคุณสามารถเล่นเกมต่อไปได้เมื่อได้รับสถานะเกม การแทนค่าสถานะเกมของคุณก็เป็นที่ยอมรับได้ แม้ว่าเราอาจต้องการรวมข้อมูลทุกประเภทในสถานะเกมของเรา แต่เราไม่อยากใส่ข้อมูลมากเกินไปเข้าไป การปรับเปลี่ยนสถานะเกมนี้เพื่อสร้างสถานะใหม่จะเป็นเรื่องเจ็บปวดจริงๆ

ตอนนี้ สำหรับสถานะเกม `TicTacToe` ของเรา การเก็บเฉพาะตำแหน่งของ X และ O ทั้งหมดจะเพียงพอที่จะแทนค่าข้อมูลเกมทั้งหมดในเวลานั้นหรือไม่? มันบอกเราว่าถึงคิวใครต่อไปหรือไม่? การดู X และ O บนกระดานและนับพวกมันน่าจะบอกเราได้ แต่นั่นหมายถึงการคำนวณเพิ่มเติม เพื่อหลีกเลี่ยงสิ่งนี้ เราจะเก็บว่าเป็นคิวของใครที่จะเดินต่อไปในสถานะเกมด้วย

คิดเกี่ยวกับสิ่งที่เราทำที่นี่ เราได้ลดการคำนวณพิเศษโดยการเก็บข้อมูลเพิ่มเติมในสถานะเกม ตอนนี้ ข้อมูลนี้อาจจะไม่จำเป็นอย่างยิ่งที่จะบอกเราเกี่ยวกับสถานะของเกม แต่มันช่วยประหยัดเวลาการคำนวณเพิ่มเติม เราจะทำเช่นนี้อีกในภายหลัง

ในการเก็บสถานะเกมเราจะใช้ namedtuple `GameState`

* `to_move`: string ของตัวอักษรตัวเดียว ไม่ว่าจะเป็น 'X' หรือ 'O'

* `utility`: 1 สำหรับชนะ, -1 สำหรับแพ้, 0 ในกรณีอื่น

* `board`: ตำแหน่งทั้งหมดของ X และ O บนกระดาน

* `moves`: การเดินทั้งหมดที่เป็นไปได้จากสถานะปัจจุบัน โปรดสังเกตที่นี่ว่า การเก็บการเดินเป็น list ดังที่ทำที่นี่ จะเพิ่มความซับซ้อนของพื้นที่ของ Minimax Search จาก `O(m)` เป็น `O(bm)` อ้างอิงถึงส่วน 5.2.1 ของหนังสือ

### การแทนค่าการเดินในเกม TicTacToe

ตอนนี้ที่เราได้ตัดสินใจแล้วว่าสถานะเกมของเราจะถูกแทนค่าอย่างไร ก็ถึงเวลาตัดสินใจว่าการเดินของเราจะถูกแทนค่าอย่างไร มันง่ายในการใช้การเดินนี้เพื่อปรับเปลี่ยนสถานะเกมปัจจุบันเพื่อสร้างสถานะใหม่

สำหรับเกม `TicTacToe` ของเรา เราจะแทนค่าการเดินด้วย tuple เพียงอันเดียว โดยที่องค์ประกอบแรกและที่สองของ tuple จะแทนค่าแถวและคอลัมน์ตามลำดับ ที่การเดินต่อไปจะถูกทำ ว่าจะทำ 'X' หรือ 'O' จะถูกตัดสินโดย `to_move` ใน namedtuple `GameState`

## เกมรูปที่ 5.2 (Fig52 Game)

สำหรับตัวอย่างที่ง่ายกว่า เราจะแทนค่าเกมใน **รูปที่ 5.2** ของหนังสือ

<img src="images/fig_5_2.png" width="75%">

สถานะถูกแทนค่าด้วยตัวอักษรพิมพ์ใหญ่ภายในสามเหลี่ยม (เช่น "A") ในขณะที่การเดินคือป้ายบนขอบระหว่างสถานะ (เช่น "a1") โหนดสิ้นสุดมีค่าประโยชน์ โปรดสังเกตว่าโหนดสิ้นสุดจะถูกตั้งชื่อในตัวอย่างนี้เป็น 'B1', 'B2' และ 'B3' สำหรับโหนดด้านล่าง 'B' และอื่นๆ

เราจะสร้างแบบจำลองการเดิน ค่าประโยชน์ และสถานะเริ่มต้นแบบนี้:

In [5]:
moves = dict(A=dict(a1='B', a2='C', a3='D'),
                 B=dict(b1='B1', b2='B2', b3='B3'),
                 C=dict(c1='C1', c2='C2', c3='C3'),
                 D=dict(d1='D1', d2='D2', d3='D3'))
utils = dict(B1=3, B2=12, B3=8, C1=2, C2=4, C3=6, D1=14, D2=5, D3=2)
initial = 'A'

ใน `moves` เรามีระบบ dictionary ที่ซ้อนกัน dictionary ภายนอกมี key เป็นสถานะและ value เป็นการเดินที่เป็นไปได้จากสถานะนั้น (เป็น dictionary) dictionary ภายในของการเดินมี key เป็นชื่อการเดินและ value เป็นสถานะถัดไปหลังจากการเดินเสร็จสิ้น

ด้านล่างเป็นตัวอย่างที่แสดง `moves` เราต้องการสถานะถัดไปหลังจากการเดิน 'a1' จาก 'A' ซึ่งคือ 'B' การดูภาพข้างบนอย่างรวดเร็วจะยืนยันว่านี่คือกรณีจริง

In [5]:
print(moves['A']['a1'])

B


ตอนนี้เราจะดูฟังก์ชันที่เราต้องดำเนินการ ก่อนอื่นเราต้องสร้าง object ของ class `Fig52Game`

In [6]:
fig52 = Fig52Game()

`actions`: ส่งคืนรายการการเดินที่สามารถทำได้จากสถานะที่กำหนด

In [ ]:
psource(Fig52Game.actions)

In [8]:
print(fig52.actions('B'))

['b1', 'b2', 'b3']


`result`: ส่งคืนสถานะถัดไปหลังจากที่เราทำการเดินเฉพาะ

In [ ]:
psource(Fig52Game.result)

In [10]:
print(fig52.result('A', 'a1'))

B


`utility`: ส่งคืนค่าของสถานะสิ้นสุดสำหรับผู้เล่น ('MAX' และ 'MIN') โปรดสังเกตว่าสำหรับ 'MIN' ค่าที่ส่งคืนเป็นค่าลบของค่าประโยชน์

In [ ]:
psource(Fig52Game.utility)

In [12]:
print(fig52.utility('B1', 'MAX'))
print(fig52.utility('B1', 'MIN'))

3
-3


`terminal_test`: ส่งคืน `True` ถ้าสถานะที่กำหนดเป็นสถานะสิ้นสุด มิฉะนั้น `False`

In [ ]:
psource(Fig52Game.terminal_test)

In [14]:
print(fig52.terminal_test('C3'))

True


`to_move`: ส่งคืนผู้เล่นที่จะเดินในสถานะนี้

In [ ]:
psource(Fig52Game.to_move)

In [16]:
print(fig52.to_move('A'))

MAX


โดยรวมแล้ว class `Fig52` ที่สืบทอดมาจาก class `Game` และเขียนฟังก์ชันของมันใหม่:

In [ ]:
psource(Fig52Game)

# MIN-MAX

## ภาพรวม (Overview)

อัลกอริทึมนี้ (มักเรียกว่า *Minimax*) คำนวณการเดินต่อไปสำหรับผู้เล่น (MIN หรือ MAX) ในสถานะปัจจุบันของพวกเขา มันคำนวณค่า minimax ของสถานะที่ตามมาแบบเรียกซ้ำ จนกว่าจะถึงสถานะสิ้นสุด (ใบของต้นไม้) โดยใช้ค่า `utility` ของสถานะสิ้นสุด มันคำนวณค่าของสถานะพาเรนต์จนกว่าจะถึงโหนดเริ่มต้น (รากของต้นไม้)

เป็นที่น่าสังเกตว่าอัลกอริทึมทำงานแบบ depth-first สามารถดู pseudocode ได้ด้านล่าง:

In [6]:
pseudocode("Minimax-Decision")

### AIMA3e
__function__ MINIMAX-DECISION(_state_) __returns__ _an action_  
&emsp;__return__ arg max<sub> _a_ &Element; ACTIONS(_s_)</sub> MIN\-VALUE(RESULT(_state_, _a_))  

---
__function__ MAX\-VALUE(_state_) __returns__ _a utility value_  
&emsp;__if__ TERMINAL\-TEST(_state_) __then return__ UTILITY(_state_)  
&emsp;_v_ &larr; &minus;&infin;  
&emsp;__for each__ _a_ __in__ ACTIONS(_state_) __do__  
&emsp;&emsp;&emsp;_v_ &larr; MAX(_v_, MIN\-VALUE(RESULT(_state_, _a_)))  
&emsp;__return__ _v_  

---
__function__ MIN\-VALUE(_state_) __returns__ _a utility value_  
&emsp;__if__ TERMINAL\-TEST(_state_) __then return__ UTILITY(_state_)  
&emsp;_v_ &larr; &infin;  
&emsp;__for each__ _a_ __in__ ACTIONS(_state_) __do__  
&emsp;&emsp;&emsp;_v_ &larr; MIN(_v_, MAX\-VALUE(RESULT(_state_, _a_)))  
&emsp;__return__ _v_  

---
__Figure__ ?? An algorithm for calculating minimax decisions. It returns the action corresponding to the best possible move, that is, the move that leads to the outcome with the best utility, under the assumption that the opponent plays to minimize utility. The functions MAX\-VALUE and MIN\-VALUE go through the whole game tree, all the way to the leaves, to determine the backed\-up value of a state. The notation argmax <sub>_a_ &Element; _S_</sub> _f_(_a_) computes the element _a_ of set _S_ that has maximum value of _f_(_a_).

---
__function__ EXPECTIMINIMAX(_s_) =     
&emsp;UTILITY(_s_) __if__ TERMINAL\-TEST(_s_)  
&emsp;max<sub>_a_</sub> EXPECTIMINIMAX(RESULT(_s, a_)) __if__ PLAYER(_s_)= MAX  
&emsp;min<sub>_a_</sub> EXPECTIMINIMAX(RESULT(_s, a_)) __if__ PLAYER(_s_)= MIN  
&emsp;∑<sub>_r_</sub> P(_r_) EXPECTIMINIMAX(RESULT(_s, r_)) __if__ PLAYER(_s_)= CHANCE

## การดำเนินงาน (Implementation)

ในการดำเนินงานเรากำลังใช้สองฟังก์ชัน `max_value` และ `min_value` เพื่อคำนวณการเดินที่ดีที่สุดสำหรับ MAX และ MIN ตามลำดับ ฟังก์ชันเหล่านี้โต้ตอบในการเรียกซ้ำสลับกัน หนึ่งเรียกอีกหนึ่งจนกว่าจะถึงสถานะสิ้นสุด เมื่อการเรียกซ้ำหยุด เราจะเหลือคะแนนสำหรับการเดินแต่ละครั้ง เราส่งคืนค่าสูงสุด แม้ว่าจะส่งคืนค่าสูงสุด มันจะทำงานสำหรับ MIN ด้วย เพราะสำหรับ MIN ค่าเหล่านั้นเป็นค่าลบของพวกมัน (ดังนั้นลำดับของค่าจะกลับด้าน ดังนั้นยิ่งสูงยิ่งดีสำหรับ MIN ด้วย)

In [7]:
psource(minimax_decision)

## ตัวอย่าง (Example)

ตอนนี้เราจะเล่นเกม Fig52 โดยใช้อัลกอริทึมนี้ ดูเกม Fig52Game จากข้างบนเพื่อติดตาม

ถึงคิว MAX ที่จะเดิน และเขาอยู่ในสถานะ A เขาสามารถไปยัง B, C หรือ D โดยใช้การเดิน a1, a2 และ a3 ตามลำดับ เป้าหมายของ MAX คือการเพิ่มค่าสุดท้ายให้ได้มากที่สุด ดังนั้น เพื่อตัดสินใจ MAX ต้องการทราบค่าที่โหนดดังกล่าวและเลือกค่าที่มากที่สุด หลังจาก MAX ถึงคิว MIN ที่จะเล่น ดังนั้น MAX ต้องการทราบว่าค่าของ B, C และ D จะเป็นเท่าใดหลังจาก MIN เล่น

ปัญหาจะกลายเป็นการเดินใดที่ MIN จะทำที่ B, C และ D สถานะตัวตามของโหนดทั้งหมดเหล่านี้เป็นสถานะสิ้นสุด ดังนั้น MIN จะเลือกค่าที่เล็กที่สุดสำหรับแต่ละโหนด ดังนั้น สำหรับ B เขาจะเลือก 3 (จากการเดิน b1), สำหรับ C เขาจะเลือก 2 (จากการเดิน c1) และสำหรับ D เขาจะเลือก 2 อีกครั้ง (จากการเดิน d3)

มาดูในโค้ด:

In [19]:
print(minimax_decision('B', fig52))
print(minimax_decision('C', fig52))
print(minimax_decision('D', fig52))

b1
c1
d3


ตอนนี้ MAX ทราบว่าค่าสำหรับ B, C และ D คือ 3, 2 และ 2 (ผลิตโดยการเดินข้างบนของ MIN) ค่าที่มากที่สุดคือ 3 ซึ่งเขาจะได้ด้วยการเดิน a1 นี่คือการเดินที่ MAX จะทำ มาดูอัลกอริทึมในการกระทำเต็มรูปแบบ:

In [20]:
print(minimax_decision('A', fig52))

a1


## การแสดงผลด้วยภาพ (Visualization)

ด้านล่างเรามีการแสดงผลเกมง่ายๆ โดยใช้อัลกอริทึม หลังจากที่คุณรันคำสั่ง ให้คลิกที่เซลล์เพื่อเดินเกมต่อไป คุณสามารถป้อนค่าของคุณเองผ่านรายการของจำนวนเต็ม 27 ตัว

In [8]:
from notebook import Canvas_minimax
from random import randint

In [9]:
minimax_viz = Canvas_minimax('minimax_viz', [randint(1, 50) for i in range(27)])

# ALPHA-BETA

## ภาพรวม (Overview)

ในขณะที่ *Minimax* เหมาะสำหรับการคำนวณการเดิน แต่มันสามารถกลายเป็นเรื่องยุ่งยากเมื่อจำนวนสถานะเกมมีมากขึ้น อัลกอริทึมต้องค้นหาใบทั้งหมดของต้นไม้ ซึ่งเพิ่มขึ้นแบบเลขชี้กำลังกับความลึกของมัน

สำหรับ Tic-Tac-Toe ที่มีความลึกของต้นไม้คือ 9 (หลังจากการเดินที่ 9 เกมจบ) เราสามารถมีสถานะสิ้นสุดได้มากที่สุด 9! (มากที่สุดเพราะไม่ใช่โหนดสิ้นสุดทั้งหมดอยู่ที่ระดับสุดท้ายของต้นไม้; บางตัวอยู่สูงขึ้นเพราะเกมจบก่อนการเดินที่ 9) นี่ไม่เลวเท่าไหร่ แต่สำหรับปัญหาที่ซับซ้อนกว่าเช่นหมากรุก เรามีโหนดสิ้นสุดมากกว่า $10^{40}$ โชคไม่ดีที่เรายังไม่พบวิธีตัดเลขชี้กำลังออกไป แต่เราพบวิธีบรรเทาภาระงาน

ที่นี่เราจะตรวจสอบการ *ตัดแต่ง (pruning)* ต้นไม้เกม ซึ่งหมายถึงการเอาส่วนที่เราไม่จำเป็นต้องตรวจสอบออกไป ประเภทของการตัดแต่งเฉพาะนี้เรียกว่า *alpha-beta* และการค้นหาโดยรวมเรียกว่า *alpha-beta search*

เพื่อแสดงส่วนใดของต้นไม้ที่เราไม่จำเป็นต้องค้นหา เราจะดูตัวอย่าง `Fig52Game`

ในตัวอย่างเกม เราต้องหาการเดินที่ดีที่สุดสำหรับผู้เล่น MAX ที่สถานะ A ซึ่งคือค่าสูงสุดของการเดินที่เป็นไปได้ของ MIN ที่สถานะตัวตาม

`MAX(A) = MAX( MIN(B), MIN(C), MIN(D) )`

`MIN(B)` คือค่าต่ำสุดของ 3, 12, 8 ซึ่งคือ 3 ดังนั้นสูตรข้างบนจะกลายเป็น:

`MAX(A) = MAX( 3, MIN(C), MIN(D) )`

การเดินต่อไปที่เราจะตรวจสอบคือ c1 ซึ่งนำไปสู่สถานะสิ้นสุดด้วยค่าประโยชน์ 2 ก่อนที่เราจะค้นหาต่อภายใต้สถานะ C มากลับไปในสูตรของเราด้วยค่าใหม่:

`MAX(A) = MAX( 3, MIN(2, c2, .... cN), MIN(D) )`

เราไม่ทราบว่าสถานะ C อนุญาตการเดินกี่ครั้ง แต่เราทราบว่าการเดินแรกได้ผลลัพธ์ค่า 2 เราต้องค้นหาต่อภายใต้ C หรือไม่? คำตอบคือไม่ ค่าที่ MIN จะเลือกใน C จะมากที่สุด 2 เนื่องจาก MAX มีตัวเลือกที่จะเลือกบางอย่างที่มากกว่านั้นแล้ว คือ 3 จาก B เขาไม่จำเป็นต้องค้นหาต่อภายใต้ C

ใน *alpha-beta* เราใช้พารามิเตอร์เพิ่มเติมสองตัวสำหรับแต่ละสถานะ/โหนด *a* และ *b* ที่อธิบายขอบเขตในการเดินที่เป็นไปได้ พารามิเตอร์ *a* แสดงตัวเลือกที่ดีที่สุด (ค่าสูงสุด) สำหรับ MAX ตามเส้นทางนั้น ในขณะที่ *b* แสดงตัวเลือกที่ดีที่สุด (ค่าต่ำสุด) สำหรับ MIN เมื่อเราดำเนินไป เราจะอัปเดต *a* และ *b* และตัดแต่งแขนงโหนดเมื่อค่าของโหนดแย่กว่าค่าของ *a* และ *b* สำหรับ MAX และ MIN ตามลำดับ

ในตัวอย่างข้างบน หลังจากการค้นหาภายใต้สถานะ B, MAX มีค่า *a* เป็น 3 ดังนั้น เมื่อค้นหาโหนด C เราพบค่าน้อยกว่านั้น คือ 2 เราจึงหยุดการค้นหาภายใต้ C

คุณสามารถอ่าน pseudocode ด้านล่าง:

In [10]:
pseudocode("Alpha-Beta-Search")

### AIMA3e
__function__ ALPHA-BETA-SEARCH(_state_) __returns__ an action  
&emsp;_v_ &larr; MAX\-VALUE(_state_, &minus;&infin;, &plus;&infin;)  
&emsp;__return__ the _action_ in ACTIONS(_state_) with value _v_  

---
__function__ MAX\-VALUE(_state_, _&alpha;_, _&beta;_) __returns__ _a utility value_  
&emsp;__if__ TERMINAL\-TEST(_state_) __then return__ UTILITY(_state_)  
&emsp;_v_ &larr; &minus;&infin;  
&emsp;__for each__ _a_ __in__ ACTIONS(_state_) __do__  
&emsp;&emsp;&emsp;_v_ &larr; MAX(_v_, MIN\-VALUE(RESULT(_state_, _a_), _&alpha;_, _&beta;_))  
&emsp;&emsp;&emsp;__if__ _v_ &ge; _&beta;_ __then return__ _v_  
&emsp;&emsp;&emsp;_&alpha;_ &larr; MAX(_&alpha;_, _v_)  
&emsp;__return__ _v_  

---
__function__ MIN\-VALUE(_state_, _&alpha;_, _&beta;_) __returns__ _a utility value_  
&emsp;__if__ TERMINAL\-TEST(_state_) __then return__ UTILITY(_state_)  
&emsp;_v_ &larr; &plus;&infin;  
&emsp;__for each__ _a_ __in__ ACTIONS(_state_) __do__  
&emsp;&emsp;&emsp;_v_ &larr; MIN(_v_, MAX\-VALUE(RESULT(_state_, _a_), _&alpha;_, _&beta;_))  
&emsp;&emsp;&emsp;__if__ _v_ &le; _&alpha;_ __then return__ _v_  
&emsp;&emsp;&emsp;_&beta;_ &larr; MIN(_&beta;_, _v_)  
&emsp;__return__ _v_  


---
__Figure__ ?? The alpha\-beta search algorithm. Notice that these routines are the same as the MINIMAX functions in Figure ??, except for the two lines in each of MIN\-VALUE and MAX\-VALUE that maintain _&alpha;_ and _&beta;_ (and the bookkeeping to pass these parameters along).

## การดำเนินงาน (Implementation)

เช่นเดียวกับ *minimax* เราใช้ฟังก์ชัน `max_value` และ `min_value` อีกครั้ง แต่คราวนี้เราใช้ค่า *a* และ *b* อัปเดตพวกมันและหยุดการเรียกแบบเรียกซ้ำหากเราไปสู่โหนดที่มีค่าแย่กว่า *a* และ *b* (สำหรับ MAX และ MIN) อัลกอริทึมหาค่าสูงสุดและส่งคืนการเดินที่ได้ผลลัพธ์นั้น

การดำเนินงาน:

In [21]:
%psource alphabeta_search

## ตัวอย่าง (Example)

เราจะเล่นเกม Fig52 ด้วยอัลกอริทึม *alpha-beta* search ถึงคิว MAX ที่จะเล่นที่สถานะ A

In [22]:
print(alphabeta_search('A', fig52))

a1


การเดินที่ดีที่สุดสำหรับ MAX คือ a1 ด้วยเหตุผลที่ให้ไว้ข้างบน MIN จะเลือกการเดิน b1 สำหรับ B ได้ผลลัพธ์ค่า 3 อัปเดตค่า *a* ของ MAX เป็น 3 จากนั้น เมื่อเราพบภายใต้ C โหนดที่มีค่า 2 เราจะหยุดการค้นหาภายใต้ sub-tree นั้นเนื่องจากมันน้อยกว่า *a* จาก D เรามีค่า 2 ดังนั้น การเดินที่ดีที่สุดสำหรับ MAX คือการที่ได้ผลลัพธ์ค่า 3 ซึ่งคือ a1

ด้านล่างเราเห็นการเดินที่ดีที่สุดสำหรับ MIN เริ่มจาก B, C และ D ตามลำดับ โปรดสังเกตว่าอัลกอริทึมในกรณีเหล่านี้ทำงานเหมือนกับ *minimax* เนื่องจากโหนดทั้งหมดด้านล่างสถานะที่กล่าวถึงข้างบนเป็นสถานะสิ้นสุด

In [23]:
print(alphabeta_search('B', fig52))
print(alphabeta_search('C', fig52))
print(alphabeta_search('D', fig52))

b1
c1
d3


## การแสดงผลด้วยภาพ (Visualization)

ด้านล่างคุณจะพบการแสดงผลด้วยภาพของอัลกอริทึม alpha-beta สำหรับเกมง่ายๆ คลิกที่เซลล์หลังจากที่คุณรันคำสั่งเพื่อเดินเกมต่อไป คุณสามารถป้อนค่าของคุณเองผ่านรายการของจำนวนเต็ม 27 ตัว

In [11]:
from notebook import Canvas_alphabeta
from random import randint

In [12]:
alphabeta_viz = Canvas_alphabeta('alphabeta_viz', [randint(1, 50) for i in range(27)])

# ผู้เล่น (PLAYERS)

ดังนั้น เราจึงจบการดำเนินงานของ class `TicTacToe` และ `Fig52Game` สิ่งที่ class เหล่านี้ทำคือการกำหนดกฎของเกม เราต้องการมากกว่านั้นเพื่อสร้าง AI ที่สามารถเล่นเกมได้จริง นี่คือจุดที่ `random_player` และ `alphabeta_player` เข้ามา

## query_player
ฟังก์ชัน `query_player` ช่วยให้คุณ คู่ต่อสู้มนุษย์ เล่นเกมได้ ฟังก์ชันนี้ต้องการ method `display` ที่จะถูกดำเนินการใน game class ของคุณ เพื่อให้สถานะเกมที่ต่อเนื่องกันสามารถแสดงบนเทอร์มินัล ทำให้ง่ายต่อการมองเห็นเกมและเล่นตามนั้น

## random_player
`random_player` เป็นฟังก์ชันที่เล่นการเดินแบบสุ่มในเกม นั่นคือทั้งหมด ไม่มีอะไรมากไปกว่านี้สำหรับตัวนี้

## alphabeta_player
`alphabeta_player` ในทางกลับกัน เรียกฟังก์ชัน `alphabeta_search` ซึ่งส่งคืนการเดินที่ดีที่สุดในสถานะเกมปัจจุบัน ดังนั้น `alphabeta_player` จึงเล่นการเดินที่ดีที่สุดเสมอเมื่อได้รับสถานะเกม สมมติว่าต้นไม้เกมมีขนาดเล็กพอที่จะค้นหาทั้งหมด

## play_game
ฟังก์ชัน `play_game` จะเป็นตัวที่จะถูกใช้จริงในการเล่นเกม คุณส่งอาร์กิวเมนต์ให้มันเป็นตัวอย่างของเกมที่คุณต้องการเล่นและผู้เล่นที่คุณต้องการในเกมนี้ ใช้มันเพื่อเล่น AI vs AI, AI vs มนุษย์ หรือแม้แต่มนุษย์ vs มนุษย์!

# มาเล่นเกมกัน! (LET'S PLAY SOME GAMES!)

## Game52

มาเริ่มต้นด้วยการทดลองกับ `Fig52Game` ก่อน สำหรับนั้นเราจะสร้างตัวอย่างของ subclass Fig52Game ที่สืบทอดมาจาก class Game:

In [13]:
game52 = Fig52Game()

ก่อนอื่นเราลองใช้ `random_player(game, state)` ของเรา เมื่อได้รับสถานะเกม มันจะให้การเดินแบบสุ่มทุกครั้ง:

In [28]:
print(random_player(game52, 'A'))
print(random_player(game52, 'A'))

a1
a3


`alphabeta_player(game, state)` จะให้การเดินที่ดีที่สุดที่เป็นไปได้เสมอ สำหรับผู้เล่นที่เกี่ยวข้อง (MAX หรือ MIN):

In [29]:
print( alphabeta_player(game52, 'A') )
print( alphabeta_player(game52, 'B') )
print( alphabeta_player(game52, 'C') )

a1
b1
c1


สิ่งที่ `alphabeta_player` ทำคือ มันเรียก method `alphabeta_full_search` อย่างง่ายๆ ทั้งสองเป็นอย่างเดียวกันโดยพื้นฐาน ในโมดูล ทั้ง `alphabeta_full_search` และ `minimax_decision` ได้ถูกดำเนินการ ทั้งสองทำงานเดียวกันและส่งคืนสิ่งเดียวกัน ซึ่งคือการเดินที่ดีที่สุดในสถานะปัจจุบัน แค่ว่า `alphabeta_full_search` มีประสิทธิภาพมากกว่าในเรื่องเวลาเพราะมันตัดแต่งต้นไม้ค้นหาและด้วยเหตุนี้ สำรวจสถานะได้น้อยกว่า

In [30]:
minimax_decision('A', game52)

'a1'

In [31]:
alphabeta_search('A', game52)

'a1'

สาธิตฟังก์ชัน play_game กับ game52:

In [32]:
game52.play_game(alphabeta_player, alphabeta_player)

B1


3

In [33]:
game52.play_game(alphabeta_player, random_player)

B2


12

In [34]:
game52.play_game(query_player, alphabeta_player)

current state:
A
available moves: ['a1', 'a2', 'a3']

Your move? a1
B1


3

In [35]:
game52.play_game(alphabeta_player, query_player)

current state:
B
available moves: ['b1', 'b2', 'b3']

Your move? b1
B1


3

โปรดสังเกตว่าหากคุณเป็นผู้เล่นคนแรก alphabeta_player จะเล่นเป็น MIN และหากคุณเป็นผู้เล่นคนที่สอง alphabeta_player จะเล่นเป็น MAX นี่เกิดขึ้นเพราะนั่นคือวิธีที่เกมถูกกำหนดใน class Fig52Game การดูโค้ดของ class นี้ควรจะทำให้ชัดเจน

## TicTacToe

ตอนนี้มาเล่น `TicTacToe` ก่อนอื่นเราเริ่มต้นเกมโดยการสร้างตัวอย่างของ subclass TicTacToe ที่สืบทอดมาจาก class Game:

In [14]:
ttt = TicTacToe()

เราสามารถพิมพ์สถานะโดยใช้ method display:

In [15]:
ttt.display(ttt.initial)

. . . 
. . . 
. . . 


อืม นั่นคือสถานะเริ่มต้นของเกม ไม่มี X และไม่มี O

มาสร้างสถานะเกมใหม่ด้วยตัวเราเองเพื่อทดลอง:

In [16]:
my_state = GameState(
    to_move = 'X',
    utility = '0',
    board = {(1,1): 'X', (1,2): 'O', (1,3): 'X',
             (2,1): 'O',             (2,3): 'O',
             (3,1): 'X',
            },
    moves = [(2,2), (3,2), (3,3)]
    )

แล้วสถานะเกมนี้มีลักษณะอย่างไร?

In [17]:
ttt.display(my_state)

X O X 
O . O 
X . . 


`random_player` จะทำตัวอย่างที่เขาควรจะเป็น กล่าวคือ *แบบสุ่มเทียม*:

In [18]:
random_player(ttt, my_state)

(2, 2)

In [19]:
random_player(ttt, my_state)

(3, 3)

แต่ `alphabeta_player` จะให้การเดินที่ดีที่สุดเสมอ ตามที่คาดหวัง:

In [20]:
alphabeta_player(ttt, my_state)

(2, 2)

ตอนนี้มาทำให้ผู้เล่นสองคนเล่นต่อกัน เราใช้ฟังก์ชัน `play_game` สำหรับสิ่งนี้ ฟังก์ชัน `play_game` ทำให้ผู้เล่นเล่นแมตช์ต่อกันและส่งคืนค่าประโยชน์สำหรับผู้เล่นแรกของสถานะสิ้นสุดที่ถึงเมื่อเกมจบ ดังนั้น สำหรับเกม `TicTacToe` ของเรา หากเราได้ผลลัพธ์ +1 ผู้เล่นคนแรกชนะ, -1 หากผู้เล่นคนที่สองชนะ และ 0 หากแมตช์จบลงด้วยการเสมอ

In [25]:
ttt.play_game(random_player, alphabeta_player)

O O X 
X O . 
X X O 


-1

ผลลัพธ์คือ (โดยปกติ) -1 เพราะ `random_player` แพ้ `alphabeta_player` อย่างไรก็ตาม บางครั้ง `random_player` สามารถเสมอกับ `alphabeta_player` ได้

เนื่องจาก `alphabeta_player` เล่นอย่างสมบูรณ์แบบ แมตช์ระหว่าง `alphabeta_player` สองตัวควรจบลงด้วยการเสมอเสมอ มาดูว่าสิ่งนี้เกิดขึ้นหรือไม่:

In [26]:
for _ in range(10):
    print(ttt.play_game(alphabeta_player, alphabeta_player))

X X O 
O O X 
X O X 
0
X X O 
O O X 
X O X 
0
X X O 
O O X 
X O X 
0
X X O 
O O X 
X O X 
0
X X O 
O O X 
X O X 
0
X X O 
O O X 
X O X 
0
X X O 
O O X 
X O X 
0
X X O 
O O X 
X O X 
0
X X O 
O O X 
X O X 
0
X X O 
O O X 
X O X 
0


`random_player` ไม่ควรชนะ `alphabeta_player` เลย มาทดสอบเรื่องนั้น

In [27]:
for _ in range(10):
    print(ttt.play_game(random_player, alphabeta_player))

. . O 
X O . 
O X X 
-1
O O O 
. X X 
O X X 
-1
X O . 
. O . 
X O X 
-1
. O . 
. O X 
X O X 
-1
O X X 
O O X 
X . O 
-1
O X O 
X O X 
X O X 
0
X O X 
O O X 
X O . 
-1
O O O 
X O X 
X . X 
-1
O X X 
O X O 
O . X 
-1
X O X 
X O X 
O O . 
-1


## Canvas_TicTacToe(Canvas)

subclass นี้ใช้เพื่อเล่นเกม TicTacToe แบบโต้ตอบใน Jupyter notebooks class TicTacToe ถูกเรียกขณะเริ่มต้น subclass นี้

มาทำแมตช์ระหว่าง `random_player` และ `alphabeta_player` คลิกที่กระดานเพื่อเรียกผู้เล่นให้ทำการเดิน

In [28]:
from notebook import Canvas_TicTacToe

In [29]:
bot_play = Canvas_TicTacToe('bot_play', 'random', 'alphabeta')

ตอนนี้ มาเล่นเกมด้วยตัวเราเองกับ `random_player`:

In [30]:
rand_play = Canvas_TicTacToe('rand_play', 'human', 'random')

ยี่ห้า! เรา (โดยปกติ) ชนะ แต่เราไม่สามารถชนะ `alphabeta_player` ได้ ไม่ว่าเราจะพยายามหนักแค่ไหน

In [31]:
ab_play = Canvas_TicTacToe('ab_play', 'human', 'alphabeta')